# Data Recommender Engine


## Pre-proccessing

In [1]:
# import
import pandas as pd 
import numpy as np

In [2]:
# import dataframe
data=pd.read_csv('../../Data/dataframe.csv')

#create copy
df=data.copy()

In [4]:
# get rid of first column since its not needed

df = df.iloc[: , 1:]

In [5]:
df.head()

,podcast_id,author_id,created_at,podcast,title,rating,content,category_arts,category_arts-design,category_arts-fashion-beauty,...,category_kids-family,category_music,category_religion-spirituality,category_society-culture,category_society-culture-personal-journals,category_society-culture-philosophy,category_society-culture-places-travel,category_spirituality,category_technology,category_tv-film
0,c61aa81c9b929a66f0c1db6cbe5d8548,F7E5A318989779D,2018-04-24T12:05:16-07:00,backstage-at-tilles-center,really interesting!,5.0,Thanks for providing these insights. Really e...,1,0,0,...,0,1,0,0,0,0,0,0,0,0
1,c61aa81c9b929a66f0c1db6cbe5d8548,F6BF5472689BD12,2018-05-09T18:14:32-07:00,backstage-at-tilles-center,Must listen for anyone interested in the arts!!!,5.0,Super excited to see this podcast grow. So man...,1,0,0,...,0,1,0,0,0,0,0,0,0,0
2,ad4f2bf69c72b8db75978423c25f379e,1AB95B8E6E1309E,2019-06-11T14:53:39-07:00,ted-talks-daily,nauseatingly left,1.0,"I'm a liberal myself, but its pretty obvious a...",1,1,0,...,0,0,0,0,0,0,0,0,0,0
3,ad4f2bf69c72b8db75978423c25f379e,11BB760AA5DEBD1,2018-05-31T13:08:09-07:00,ted-talks-daily,Diverse stories,5.0,I find Tedx talks very inspirational but I oft...,1,1,0,...,0,0,0,0,0,0,0,0,0,0
4,ad4f2bf69c72b8db75978423c25f379e,D86032C8E57D15A,2019-06-19T13:56:05-07:00,ted-talks-daily,👍👍👍👍,5.0,"I love this podcast, it is so good.",1,1,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
# de-dummies variables
# all the category variables were turned into dummy-variables in order to aggregate the duplicate reviews
# they now need to be turned back into categorical variables

#create new column for which the aggregated
df['tags']=''

# takes dummmies and puts them in the new column
for col_name in df.columns[7:]:
    df.loc[df[col_name]==1,'tags']= df['tags']+' '+col_name

# get rid of columns with dummy variables
df=df[['podcast_id', 'author_id', 'created_at', 'podcast', 'title', 'rating', #gets rid of dummie features
       'content','tags']]


# slice strings to get rid of "category_" part of the value

df['tags']=df['tags'].astype(str)

df['tags']=df['tags'].apply(lambda x: [w[9:] for w in x.split()])

In [7]:
# change date+time value in in the 'created_at' column into 'year'

# change to datetime
df['created_at']=pd.to_datetime(df['created_at'])

# change datetime to year
df['year']=df['created_at'].apply(lambda x: x.strftime('%Y'))

In [9]:
# turn rating into string 
df['rating'] = df.rating.astype(str)

In [10]:
# pre-processsing content

#installing all packages
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer 
from nltk.tokenize.treebank import TreebankWordDetokenizer
import nltk
nltk.download('wordnet')

# pseudo-pipeline
def cleaning_content(x):
    
    # tokenize and remove punctuation
    tokenizer = RegexpTokenizer(r'\w+')
    tokenized=tokenizer.tokenize(x)
    
    # turn all characters into lowercase
    lowered = [w.lower() for w in tokenized]
    
    #removing stopwords
    stop_words = stopwords.words('english')
    rm_stopwords=[w for w in lowered if not w in stop_words]
    
    #lemmatize
    lemmatizer = WordNetLemmatizer()  
    lemmatized=[lemmatizer.lemmatize(token) for token in rm_stopwords]

    return lemmatized

df['lemmatized']=[cleaning_content(x) for x in df['content']]



[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\lazar\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [11]:
# keep only necessary rows
df=df[['podcast_id','author_id','podcast','year','rating','lemmatized','tags']]

# turn rating into string 
df['rating'] = df.rating.astype(str)

C:\Users\lazar\AppData\Local\Temp/ipykernel_9684/2586138558.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['rating'] = df.rating.astype(str)


In [12]:
# create a soup column which combines all the features
# since the dataframe has some columns which are strings and others which are lists of strings, addditional processing needs to be performed

# combining all the stringed columns into a column which will be called 'soup'
df['string_soup']=df['author_id']+' '+df['year']+' '+df['podcast']+' '+df['rating']

# this soup column will not have the rating included. Later will be testing whether changes in parameters will create better results
df['string_soup_r']=df['author_id']+' '+df['year']+' '+df['podcast']


# combine all lists 
def extender(x):
    return ' '.join(x['tags'] + x['lemmatized'])
df['lst_soup']=df.apply(extender, axis=1)


# combine both columns into one soup
def extender_soup(x):
    return x['string_soup'] + ' ' + x['lst_soup']
df['soup']=df.apply(extender_soup, axis=1)

# combine both columns into one soup wihout rating
def extender_soup(x):
    return x['string_soup_r'] + ' ' + x['lst_soup']
df['soup_r']=df.apply(extender_soup, axis=1)

C:\Users\lazar\AppData\Local\Temp/ipykernel_9684/3302368480.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['string_soup']=df['author_id']+' '+df['year']+' '+df['podcast']+' '+df['rating']
C:\Users\lazar\AppData\Local\Temp/ipykernel_9684/3302368480.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['string_soup_r']=df['author_id']+' '+df['year']+' '+df['podcast']


In [13]:
# making copy of dataframe

df_soup=df.copy()
df_soup_r=df.copy() #this will be the new dataframe with rating as it's own column

# dropping all columns except for the soup

df_soup=df_soup[['podcast_id','soup']] 
df_soup_r=df_soup_r[['podcast_id','soup','rating']]

In [14]:
df_soup_r.head()

,podcast_id,soup,rating
0,c61aa81c9b929a66f0c1db6cbe5d8548,F7E5A318989779D 2018 backstage-at-tilles-cente...,5.0
1,c61aa81c9b929a66f0c1db6cbe5d8548,F6BF5472689BD12 2018 backstage-at-tilles-cente...,5.0
2,ad4f2bf69c72b8db75978423c25f379e,1AB95B8E6E1309E 2019 ted-talks-daily 1.0 arts ...,1.0
3,ad4f2bf69c72b8db75978423c25f379e,11BB760AA5DEBD1 2018 ted-talks-daily 5.0 arts ...,5.0
4,ad4f2bf69c72b8db75978423c25f379e,D86032C8E57D15A 2019 ted-talks-daily 5.0 arts ...,5.0


In [15]:
df_soup.head()

,podcast_id,soup
0,c61aa81c9b929a66f0c1db6cbe5d8548,F7E5A318989779D 2018 backstage-at-tilles-cente...
1,c61aa81c9b929a66f0c1db6cbe5d8548,F6BF5472689BD12 2018 backstage-at-tilles-cente...
2,ad4f2bf69c72b8db75978423c25f379e,1AB95B8E6E1309E 2019 ted-talks-daily 1.0 arts ...
3,ad4f2bf69c72b8db75978423c25f379e,11BB760AA5DEBD1 2018 ted-talks-daily 5.0 arts ...
4,ad4f2bf69c72b8db75978423c25f379e,D86032C8E57D15A 2019 ted-talks-daily 5.0 arts ...


## Models

### Cosine Similarity

In [16]:
# Import CountVectorizer and create the count matrix
from sklearn.feature_extraction.text import CountVectorizer

# Run Vectorizer
count = CountVectorizer(max_features=5000)
count_matrix = count.fit_transform(df_soup['soup'])
count_matrix.shape

(983900, 5000)

In [17]:
count_matrix

<983900x5000 sparse matrix of type '<class 'numpy.int64'>'
	with 23308599 stored elements in Compressed Sparse Row format>

In [18]:
#turn matrix into dataframe
df_countmatrix=pd.DataFrame(count_matrix, columns=['vectors'])

#merge podcast_id and countmatrix
df_countmatrix=df_soup[['podcast_id']].join(df_countmatrix)
df_countmatrix.head()

,podcast_id,vectors
0,c61aa81c9b929a66f0c1db6cbe5d8548,"(0, 30)\t1\n (0, 343)\t1\n (0, 756)\t1\n ..."
1,c61aa81c9b929a66f0c1db6cbe5d8548,"(0, 30)\t1\n (0, 343)\t1\n (0, 756)\t1\n ..."
2,ad4f2bf69c72b8db75978423c25f379e,"(0, 325)\t2\n (0, 3950)\t1\n (0, 31)\t1\n ..."
3,ad4f2bf69c72b8db75978423c25f379e,"(0, 30)\t1\n (0, 325)\t2\n (0, 4393)\t1\n ..."
4,ad4f2bf69c72b8db75978423c25f379e,"(0, 325)\t2\n (0, 3389)\t1\n (0, 31)\t1\n ..."


In [19]:
# import sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import cosine_similarity

def recommend_podcast_cosine(df, index):
    smin = None
    imax2 = None
    row = df.iloc[index]
    
    # loop through dataframe rows
    for index2, row2 in df.iterrows():
    
        #loop through column in those rows
        #index != index2:
        if row['podcast_id'] != row2['podcast_id']: #index != index2:
            s = cosine_similarity(row['vectors'], row2['vectors'])
            
            # find smallest distance between two vectors
            if smin == None or s < smin:
                smin = s
                imax2 = index2
    return imax2, smin

In [26]:
# results

# index of the podcast_id
i_cos = 12345 

# run the funcion
rec_cos = recommend_podcast_cosine(df_countmatrix,i_cos)
x=rec_cos[0]

In [27]:
#this this is the 'listened to' podcast
print(list(df_soup.iloc[i_cos]))

['e259af857dea4eb1fe6e3be818488bbc', 'F34A21A65DD8237 2018 hallowed-histories 5.0 society-culture society-culture-places-travel podcast present various topic folklore highly engaging way always learn something new listened highly recommend podcast']


In [28]:
# this is the recommended podcast
print(list(df_soup.iloc[x]))

['ad4f2bf69c72b8db75978423c25f379e', '1AB95B8E6E1309E 2019 ted-talks-daily 1.0 arts arts-design education liberal pretty obvious annoying trying push belief listener get politically diverse production staff like see issue multiple point view']


As we can see above, the cosine similarity model does not produce the best results.

### Jaccard Similarity

#### Jaccard Similarity

In [21]:
# function to create jaccard simualirty score
# there is no prebuilt jaccard function that takes both float points and strings, so I created my own

def jaccard_similarity(a, b): 
    # convert to set and tokenize soup
    a1 = set(a.split()) ####  can remove this and tokenize df_soup['soup']
    b1 = set(b.split())
    
    # calucate jaccard similarity
    j = float(len(a1.intersection(b1))) / len(a1.union(b1))
    return j

In [22]:
# Return recommended Podcast ID for input index of a review
def recommend_podcast_jaccard(df, index):
    smax = 0
    row = df.iloc[index]
    for index2, row2 in df.iterrows():
    # print(index, row['podcast_id'], row['soup'])
    
    #loop through column in those rows and get the cosine similarity 
        #index != index2:
        if row['podcast_id'] != row2['podcast_id']: #index != index2:
            s = jaccard_similarity(row['soup'], row2['soup'])
            
            #find greatest jaccard scores
            if s > smax:
                smax = s
                imax2 = index2
    return imax2, smax

In [23]:
# results

# index of the podcast_id
i = 12345 

# run the funcion
imax2 = recommend_podcast_jaccard(df_soup, i)
x=imax2[0]

#this this is the 'listened to' podcast
print(list(df_soup.iloc[i]))

# this is the recommended podcast
print(list(df_soup.iloc[x]))

['e259af857dea4eb1fe6e3be818488bbc', 'F34A21A65DD8237 2018 hallowed-histories 5.0 society-culture society-culture-places-travel podcast present various topic folklore highly engaging way always learn something new listened highly recommend podcast']
['f7a663723c114efc208ba3894b680b62', 'F34A21A65DD8237 2018 early-stuart-england 5.0 society-culture podcast present history early stuart england engaging way always learn something new listened highly recommend show well worth listen']


In [25]:
# print of the similarity score

imax2[1]

0.5172413793103449

Interpretation of results: Jaccard similarity tends to lead to better results compared to cosine similarity. Unlike other machine learning models, there is no true way to test the accuracy of the results. The best we can do is test several different podcasts ourselves, and decide which model performs better. 

In a proper business setting, we would use an A/B testing method. Half of the users would see the recommended podcasts from the cosine similarity model and the other half would see podcasts from the Jaccard similarity model. We would take measures to ensure both samples are representative of the whole population of users of the app/website. This is to eliminate as much bias as possible from our experiment design. We would measure the success of a given model based on a single metric or an aggregation of several metrics such as the number of clicks, listen time of the recommended podcast, or level of interaction with the podcast. Finally, we would perform a student's t-test or a similar test. 

#### Jaccard score with weighted rating

In [ ]:
# Return recommended Podcast ID for input index of a review with a weighted rating
# instead of including it within the jaccard score, the was multiplied by the jaccard score

def recommend_podcast_jaccard_rating(df, index):
    smax = 0
    row = df.iloc[index]
    for index2, row2 in df.iterrows():
        #print(index, row['podcast_id'], row['soup'])
    
    #loop through column in those rows and get the jaccard score
    #index != index2:
        if row['podcast_id'] != row2['podcast_id']:
            s = jaccard_similarity(row['soup'], row2['soup'])
            
            #weighted rating score
            s = float(s) * float(row2['rating'])
            
            if s > smax:
                smax = s
                imax2 = index2
    return imax2, smax

In [ ]:
# no real difference in results

i = 1234
imax2 = recommend_podcast_jaccard_rating(df_countmatrix,i)
x=imax2[0]
print(list(df_soup_r.iloc[i]))
print(list(df_soup_r.iloc[x]))